In [1]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
sys.path.insert(0, os.path.join(spark_home, 'python'))
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.1
      /_/

Using Python version 3.6.3 (default, Oct 13 2017 12:02:49)
SparkSession available as 'spark'.


In [2]:
import shutil
import re
import itertools
import time
from pyspark import SparkConf, SparkContext
filename="soc-data.txt"
#filename="soc-data.txt"
output_dir="soc_output"
# conf = SparkConf()
# sc = SparkContext(conf=conf)

In [3]:
start=time.time()
lines = sc.textFile(filename)
user_id = lines.map(lambda l:re.split(r'[^\w]+', l))
connection_user=user_id.flatMap(lambda u:[((u[0],i),-100) for i in u[1:]])
connection_friend=user_id.flatMap(lambda u:[(i,1) for i in itertools.combinations(u[1:], 2)])
count=connection_user.union(connection_friend).reduceByKey(lambda n1, n2: n1 + n2)
count_groupby=count.map(lambda x:(x[0][0],(x[0][1],x[1]))).groupByKey()
def gettopN(rec,topN):
    x=[]
    for i in sorted(rec[1],key=lambda k: k[1],reverse=True):
        if len(x)>=topN:
            break
        if i[1]>0:
            x.append(i[0])
    return int(rec[0]),x
result=count_groupby.map(lambda x: gettopN(x,10)).sortByKey()
#result=result.sortByKey(keyfunc=int) not working, why? 
if os.path.isdir(output_dir):
    shutil.rmtree(output_dir)
result.saveAsTextFile(output_dir)
end=time.time()
print(end-start)

173.3480544090271


testing

In [202]:
sorted(result.collect(),key=lambda i : int(i[0]))

[('0',
  ['38737',
   '27383',
   '34211',
   '31337',
   '49678',
   '12561',
   '41628',
   '12143',
   '25212',
   '33786']),
 ('1',
  ['35621',
   '35630',
   '15356',
   '25228',
   '15175',
   '25199',
   '27549',
   '9194',
   '32968',
   '16894']),
 ('2',
  ['41087',
   '3233',
   '13832',
   '19091',
   '43149',
   '112',
   '11165',
   '23597',
   '24568',
   '42704']),
 ('3',
  ['27679',
   '30',
   '85',
   '21429',
   '27571',
   '35179',
   '27617',
   '29',
   '27553',
   '27585']),
 ('4',
  ['19', '12562', '19032', '42736', '30', '85', '42748', '19009', '16', '83']),
 ('5',
  ['31232',
   '34394',
   '44887',
   '28193',
   '29724',
   '8715',
   '21556',
   '30691',
   '35605',
   '35678']),
 ('6',
  ['18581',
   '38734',
   '9841',
   '9784',
   '31',
   '17888',
   '9777',
   '35684',
   '35698',
   '63']),
 ('7', ['15', '18', '23', '25', '30', '31', '32', '36', '42', '43']),
 ('8', ['12', '24', '28', '16', '34211', '74', '77', '27', '89', '14']),
 ('9', ['82', '4364

In [140]:
for i in count_groupby.map(lambda rec: sorted(rec[1],key=lambda k:k[1],reverse=True)).take(10):
    print(i)
    print("\n")

[('19', 2), ('12562', 2), ('19032', 2), ('42736', 2), ('30', 2), ('85', 2), ('42748', 2), ('19009', 2), ('16', 2), ('83', 2), ('19058', 2), ('42742', 2), ('13', 2), ('38', 2), ('46', 2), ('89', 2), ('125', 2), ('9', 1), ('17', 1), ('21', 1), ('22', 1), ('29', 1), ('33', 1), ('34', 1), ('45', 1), ('48', 1), ('63', 1), ('66', 1), ('68', 1), ('69', 1), ('73', 1), ('84', 1), ('91', 1), ('93', 1), ('2406', 1), ('12324', 1), ('19555', 1), ('38745', 1), ('40528', 1), ('40801', 1), ('19022', 1), ('19041', 1), ('19088', 1), ('19212', 1), ('14043', 1), ('10652', 1), ('38375', 1), ('8768', 1), ('12333', 1), ('17438', 1), ('20441', 1), ('23207', 1), ('42692', 1), ('39169', 1), ('42713', 1), ('42719', 1), ('46263', 1), ('42717', 1), ('42750', 1), ('32193', 1), ('46163', 1), ('2084', 1), ('13915', 1), ('21162', 1), ('25182', 1), ('27597', 1), ('32347', 1), ('35590', 1), ('35992', 1), ('43004', 1), ('44943', 1), ('6', 1), ('7', 1), ('23', 1), ('25', 1), ('31', 1), ('32', 1), ('36', 1), ('42', 1), ('4

In [175]:
reccc=count_groupby.collect()[0]
# for i in sorted(reccc[1],key=lambda k: k[1],reverse=True):
#     print(i[0])
gettopN(reccc,10)

('4',
 ['19', '12562', '19032', '42736', '30', '85', '42748', '19009', '16', '83'])

In [211]:
def gettopN(rec,topN):
    x=[]
    for i in sorted(rec[1],key=lambda k: k[1],reverse=True):
        if len(x)>=topN:
            break
        if i[1]>0:
            x.append(i[0])
    return rec[0],tuple(x)

In [212]:
for i in count_groupby.map(lambda x: gettopN(x,10)).collect():
    print(i)

('4', ('19', '12562', '19032', '42736', '30', '85', '42748', '19009', '16', '83'))
('10', ('83', '18', '38', '89', '29', '25200', '29586', '85', '12636', '53'))
('12', ('28', '24', '34211', '25195', '27585', '27597', '27552', '25186', '27590', '22'))
('16', ('28', '17', '24', '29', '85', '29581', '12562', '34211', '12566', '67'))
('20', ('44846', '30257', '161', '29868', '43162', '14209', '13840', '29767', '166', '38737'))
('24', ('34211', '89', '29', '67', '46', '33', '34', '45', '48', '63'))
('26', ('20016', '53', '6152', '28', '29', '33', '34', '45', '48', '63'))
('40', ('45', '48', '63', '66', '68', '69', '73', '84', '91', '93'))
('44', ('45', '48', '63', '66', '68', '69', '73', '84', '91', '93'))
('8932', ('27536', '38626', '14005', '23594', '13828', '44891', '35595', '14064', '49226', '12655'))
('10623', ('14005', '29791', '35613', '4999', '13912', '34418', '34420', '45054', '12347', '13840'))
('12846', ('31232', '29791', '28193', '29724', '35589', '44846', '22939', '30691', '343

('21500', ('24933', '28613', '31153', '38062', '38746', '42155', '21160', '31300', '25801', '30147'))
('25801', ('28613', '31153', '38062', '38746', '42155', '21160', '31300', '30147', '40446', '49985'))
('17498', ('29841', '35245', '35632', '12822', '18573', '19457', '23718', '23685', '20470', '17512'))
('27548', ('27573', '29576', '30785', '32534', '35588', '41824', '43121', '27551', '27605', '30220'))
('35592', ('41824', '43121', '39060', '35646', '49566', '35713'))
('35646', ('41824', '43121', '39060', '49566', '35713'))
('17970', ('10043', '40748', '23266', '41918', '37719', '41816', '41921'))
('37719', ('41900', '44132', '44199', '41642', '44191', '41851', '44201', '41903', '40748', '44209'))
('41816', ('37254', '41921', '40748', '41886', '44057', '44063', '37346', '41833', '44052', '39268'))
('5376', ('12784', '15358', '5640', '19502', '30837', '32685', '37830', '38482', '43977', '32738'))
('9658', ('12784', '15358', '5640', '19502', '30837', '32685', '37830', '38482', '43977', 

('10382', ('13034', '14125', '19138', '22946', '23685', '27583', '33635', '33637', '34352', '41495'))
('41495', ('49862', '47180'))
('23256', ('44061', '23259', '23257', '23265', '23266', '23270', '23273', '23275', '23276', '23282'))
('23273', ('39247', '44079', '39236', '44070', '44057', '44063', '39240', '39234', '44089', '44132'))
('23282', ('23284', '23290', '23292', '23283', '23285', '23286', '23295', '23287', '23293', '23288'))
('23284', ('23290', '23292', '36674', '18963', '36978', '41851', '41940', '44111', '36855', '37178'))
('37660', ('38509', '40748', '41642', '44061', '44191', '37442', '37815', '38576', '39247', '37462'))
('38509', ('40748', '38576', '39267', '37254', '39240', '39310', '37356', '44077', '41642', '44061'))
('37442', ('40748', '37454', '37701', '37448', '37557', '37815', '38576', '37446', '37453', '37486'))
('6501', ('13930', '14152', '14234', '19063', '19138', '23553', '32505', '32507', '34352', '35586'))
('35586', ('35635', '35685', '35256', '35595', '35728

('22778', ('22780', '22790', '22791', '22795', '22779', '22782', '22792', '22801', '44795', '22789'))
('22789', ('22790', '22791', '22795', '22792', '22801', '44795', '22796', '22804', '22806', '22809'))
('22794', ('22795', '22786', '22800', '22801', '22802', '22815', '44795', '22796', '1100', '22804'))
('22796', ('22801', '44795', '22804', '22806', '22809', '22786', '22800', '22802', '22815', '1100'))
('22830', ('26341', '22856', '22867', '35094', '39014', '42815', '39647'))
('40425', ('47180', '44892', '40472', '44895', '46041', '40443', '40819', '43728', '47024', '41746'))
('30439', ('30787',))
('20726', ('12667', '15757', '22987', '23697', '33634', '33636', '36687', '37170', '2660', '3879'))
('22963', ('12667', '15757', '22987', '23697', '33634', '33636', '36687', '37170', '2660', '3879'))
('23683', ('23697', '33634', '33636', '36687', '37170', '23698', '36670', '23696', '30458', '43064'))
('30458', ('33634', '33636', '36687', '37170', '36670', '43064', '33635', '33637', '34190', '

('6934', ('11542', '16486', '20770', '23993', '39548', '40713', '45444', '10644', '23880', '40562'))
('16486', ('20770', '23993', '39548', '40713', '45444', '23880', '40562', '40605', '40675', '41076'))
('20770', ('23993', '39548', '40713', '45444', '23880', '40562', '40605', '40675', '41076', '23506'))
('40200', ('41457', '40201', '40205', '40206', '40215', '40217', '40218', '41407', '49888', '40207'))
('15362', ('25184', '29780', '43149', '26872', '43175', '44959', '25027', '43452', '19306', '16896'))
('15723', ('15744', '22505'))
('15744', ('22505',))
('2823', ('11897', '16686', '22505', '8101', '8107', '11893', '12712', '14339', '16510', '16684'))
('11897', ('16686', '22505', '12712', '14339', '16510', '16684', '1617', '23741', '46144', '11900'))
('16686', ('22505', '1617', '23741', '46144', '45022', '49280'))
('41233', ('44173', '37188', '41903', '37356', '44111', '44178', '36794', '37289', '37394', '44074'))
('44173', ('39332', '37188', '44178', '44189', '39365', '39366', '44199'